In [60]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

In [61]:
df = pd.read_csv('gb-postal-codes.csv')

In [62]:
df.head()

,postal_code,postal_code_prefix
0,PH44,PH
1,LE13,LE
2,LE14,LE
3,WR11,WR
4,BA8,BA


In [63]:
# We use Edge as a browser, set to headless mode
edge_option = webdriver.EdgeOptions()
edge_option.use_chromium = True

# edge_option.add_argument("headless") # uncomment this line if you want to see the browser window

s = Service('./msedgedriver.exe')
browser = webdriver.Edge(service=s, options=edge_option)
browser.create_options()

In [64]:
# common root used to build the url according to the postal code
url_root = 'https://www.home.co.uk/guides/house_prices.htm?location='

In [65]:
# one sub dataframe will be created for each type of housing
type_list = ['Flat','Terraced','Semi-detached','Detached']
df_types = []
for types in type_list:
    df_types.append(pd.DataFrame(columns=['postal_code', 'number', 'average_price', 'median_price']))

In [66]:
def parse_and_get(postal_code):
    # rest to not overload the website
    time.sleep(10)
    url = url_root+str(postal_code)
    browser.get(url)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    # in case of an webpage error, add a row with void caracteristics
    error = soup.find_all('h1')
    if str(error[0]) == '<h1>Website Error</h1>':
        for i in range(len(type_list)):
            df_types[i].loc[df_types[i].shape[0]] = [postal_code, '', '', '']
    else:
        # find the useful information
        tables = soup.find_all(class_='table table--plain indented-left')
        res = (str(tables[3]).split('<tr>')[2:])
        # add it to every dataframe according to the housing type
        for i in range(len(type_list)):
            line = res[i].split('</td>')
            number = int(line[1].strip('\n<td>'))
            average = str(line[2].strip('\n<td>£').strip(','))
            median = str(line[3].strip('\n<td>£').strip(','))
            df_types[i].loc[df_types[i].shape[0]] = [postal_code, number, average, median]

In [67]:
# apply the precedent function to the row of postal codes
df['postal_code'].apply(parse_and_get)

KeyboardInterrupt: 

In [ ]:
# close the browser and write the csv's

for i in range(len(df_types)):
    df_types[i].to_csv('gb-'+type_list[i].lower+'.csv')
browser.quit()